To create, train and run the model clear output of all cells and then run them all.
Make sure that all the required libraries in the cell below have already been installed on yor system. If not install them using "pip install <Name of the library>"

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.models import Sequential


tensorflow contains Sequential which has three convolution blocks i.e a three layer CNN

In [3]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [4]:
batch_size = 32
img_height = 180
img_width = 180

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [7]:
class_names = ['daisy','dandelion','roses','sunflowers','tulips']

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

rgb values are in the range [0,255] which is not good for neural networks so we normalize the values by dividing them with 255 so they come in the range [0,1]

In [9]:
normalization_layer = layers.Rescaling(1./255)

In [10]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
0.0 0.9955231


In [11]:
num_classes = len(class_names)

In [12]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [13]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

The model created above was first tested without data augmentation, it lead to overfitting because of less number of training data, so data augmentation has been adopted which creates new data samples from the existing ones by randomly changing some parameters so that e can get more testing data.

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

the code below trains the model using training dataset i.e train_ds and then validates it using the validation dataset i.e val_ds

In [15]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
92/92 [==============================] - 40s 382ms/step - loss: 1.3337 - accuracy: 0.4213 - val_loss: 1.2375 - val_accuracy: 0.5027
Epoch 2/15
92/92 [==============================] - 34s 365ms/step - loss: 1.0554 - accuracy: 0.5794 - val_loss: 1.0552 - val_accuracy: 0.6131
Epoch 3/15
92/92 [==============================] - 32s 350ms/step - loss: 0.9303 - accuracy: 0.6393 - val_loss: 0.8905 - val_accuracy: 0.6567
Epoch 4/15
92/92 [==============================] - 33s 355ms/step - loss: 0.8773 - accuracy: 0.6516 - val_loss: 1.0045 - val_accuracy: 0.6349
Epoch 5/15
92/92 [==============================] - 32s 344ms/step - loss: 0.7861 - accuracy: 0.6941 - val_loss: 0.8937 - val_accuracy: 0.6785
Epoch 6/15
92/92 [==============================] - 32s 348ms/step - loss: 0.7496 - accuracy: 0.7081 - val_loss: 0.8798 - val_accuracy: 0.6826
Epoch 7/15
92/92 [==============================] - 33s 355ms/step - loss: 0.6983 - accuracy: 0.7367 - val_loss: 0.9531 - val_accuracy: 0.6567

now that the model has been trained, it can be tested with an image that is not in the testing or validation set.

To test it with new images download images from the internet save it in the same directory as the jupyter file and pass its name as first argument to tf.keras.load_img(). Next it prints to the console the class of the flower and percent confidence in its prediction.

In [32]:
img = tf.keras.utils.load_img(
    'tulip.jpg', target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 29ms/step
This image most likely belongs to tulips with a 99.98 percent confidence.
